# Model-Free Dirac Delta Integration for Implied Volatility

Author: Zhenyu Cui, Justin Kirkbyb, Duy Nguyenc and Stephen Taylord

DOI: [10.13140/RG.2.2.21031.29602](http://dx.doi.org/10.13140/RG.2.2.21031.29602)

From the paper:

"The method is based on the novel use of the Dirac Delta function, corresponding delta sequences, and the change of variable technique. The formula is expressed through either a limit or as an infinite series of elementary functions, and we establish that the proposed formula converges to the true implied volatility value. In numerical experiments , we verify the convergence of the formula, and consider several benchmark cases, for which the data generating processes are respectively the stochastic volatility inspired (SVI) model, the stochastic alpha beta rho (SABR) model. We also establish an explicit formula for the implied volatility expressed directly in terms of respective model parameters , and use the Heston model to illustrate this idea. The delta sequence and change of variable technique that we develop is of independent interest and can be used to solve inverse problems arising in other applications."

## Formula
\begin{equation}
    \sigma_{BS}(C) = \lim_{\epsilon \rightarrow 0} \frac{S_0 \sqrt{T}}{2 \pi \sqrt{2 \epsilon}} \cdot \int_{\sigma_l}^{\sigma_u} s \cdot \exp \left( -\frac{\left(f_{BS}(s)-C\right)^2}{4\epsilon} -\frac{d_1^2(s)}{2}\right) ds\\
\end{equation}
where $\sigma_l$ and $\sigma_u$ are the upper bound and lower bound that contains the true $\sigma_{BS}$ which can be narrow down using the following formulas (Tehranchi 2016.Uniform bounds for Black-Scholes implied volatility. SIAM Journal on Financial Mathematics)
\begin{equation}
\sigma_u = \frac{-2}{\sqrt{T}}\mathcal{N}^{-1}\left(\frac{1-c}{1+e^k}\right)\\
\sigma_l = \begin{cases}
\frac{-2}{\sqrt{T}}\mathcal{N}^{-1}\left(\frac{1-c}{2e^k}\right)\quad if\space k<0\\
\frac{-2}{\sqrt{T}}\mathcal{N}^{-1}\left(\frac{1-c}{2}\right)\quad if\space k\geq 0
\end{cases}
\end{equation}
Note that this formula is for call option only, but can be used for put options by put-call parity.

In [1]:
import numpy as np
import scipy as sp
import scipy.stats
import scipy.integrate

In [2]:
# BS formulas
def d1_fun(S,K,T,r,q,sigma):
    return (np.log(S/K)+(r+0.5*sigma**2)*T)/sigma/np.sqrt(T)

def d2_fun(S,K,T,r,q,sigma):
    return (np.log(S/K)+(r-0.5*sigma**2)*T)/sigma/np.sqrt(T)

def black_scholes_call(S,K,T,r,q,sigma):
    forward = S*np.exp((r-q)*T)
    discount_factor = np.exp(-r*T)
    d1 = d1_fun(S,K,T,r,q,sigma)
    d2 = d1-sigma*np.sqrt(T)
    N = lambda x: sp.stats.norm.cdf(x)
    return np.exp(-q*T)*S*N(d1)-np.exp(-r*T)*K*N(d2)

def black_scholes_put(S,K,T,r,q,sigma):
    forward = S*np.exp((r-q)*T)
    discount_factor = np.exp(-r*T)
    d1 = d1_fun(S,K,T,r,q,sigma)
    d2 = d1-sigma*np.sqrt(T)
    N = lambda x: sp.stats.norm.cdf(x)
    return -np.exp(-q*T)*S*N(-d1)+np.exp(-r*T)*K*N(-d2)

# IV Boundary
def c_fun(C,S,T,q):
    return C/(S*np.exp(-q*T))

def k_fun(S,K,T,r,q):
    return np.log(K/(S*np.exp((r-q)*T)))

def sigma_bounds_call(C,S,K,T,r,q):
    """The bounds for call options"""
    c = c_fun(C,S,T,q)
    k = k_fun(S,K,T,r,q)
    invN = lambda x: sp.stats.norm.ppf(x)
    sigma_upper = -2/np.sqrt(T)*invN((1-c)/(1+np.exp(k)))
    if k<0:
        sigma_lower = -2/np.sqrt(T)*invN((1-c)/(2*np.exp(k)))
    else:
        sigma_lower = -2/np.sqrt(T)*invN((1-c)/(2))
    return sigma_lower,sigma_upper

def sigma_bounds_put(P,S,K,T,r,q):
    """The bounds for put options"""
    C = P + S - np.exp(-r*T)*K
    c = c_fun(C,S,T,q)
    k = k_fun(S,K,T,r,q)
    invN = lambda x: sp.stats.norm.ppf(x)
    sigma_upper = -2/np.sqrt(T)*invN((1-c)/(1+np.exp(k)))
    if k<0:
        print((1-c)/(2*np.exp(k)))
        sigma_lower = -2/np.sqrt(T)*invN((1-c)/(2*np.exp(k)))
    else:
        sigma_lower = -2/np.sqrt(T)*invN((1-c)/(2))
    return sigma_lower,sigma_upper

def dirac_delta_implied_vol(C,S,K,T,r,q,sigma_lower,sigma_upper,epsilon):
    def integral_fun(s):
        return s*np.exp(-(black_scholes_call(S,K,T,r,q,s)-C)**2/4/epsilon-(d1_fun(S,K,T,r,q,s)**2/2))
    coef = S*np.sqrt(T)/(2*np.pi*np.sqrt(2*epsilon))
    integral,err = sp.integrate.quad(integral_fun,sigma_lower,sigma_upper)
    return coef*integral

In [3]:
K_range = np.array([100 + 10*k for k in range(11)])
T_range = np.array([1/48,1/12,3/12,6/12,1])
r_range = np.array([-0.01,0,0.01,0.05])
q_range = np.array([0,0.01,0.05,0.1])
sigma_range = np.array([0.1,0.3,0.5,0.7])


## Have a Try
Run the following cell, it would test a random market case each time.

In [7]:
K = np.random.choice(K_range)
T = np.random.choice(T_range)
r = np.random.choice(r_range)
q = np.random.choice(q_range)
sigma = np.random.choice(sigma_range)

print('###################################')
print('Static Parameters')
print('###################################')
print('sigma: {}'.format(sigma))
print('r:     {}'.format(r))
print('q:     {}'.format(q))
print('K:     {}'.format(K))
print('T:     {}'.format(T))
print('###################################')
for moneyness in [0.4,0.7,0.8,0.9,1,1.1,1.2,1.3,1.6]:
    print('$$$$$$$$$$$$$$$$$$$$$')
    print('Moneyness S/K={}'.format(moneyness))
    print('$$$$$$$$$$$$$$$$$$$$$')
    S = K*moneyness
    call_price = black_scholes_call(S,K,T,r,q,sigma)
    put_price = black_scholes_put(S,K,T,r,q,sigma)
    call_sigma_bounds = sigma_bounds_call(call_price,S,K,T,r,q)
    put_sigma_bounds = sigma_bounds_put(put_price,S,K,T,r,q)
    print('S:     {:.2f}'.format(S))
    print('call:  {:.2f}'.format(call_price))
    print('put:   {:.2f}'.format(put_price))
    print('lower_vol: {}'.format(call_sigma_bounds[0]))
    print('upper_vol: {}'.format(call_sigma_bounds[1]))
    for epsilon in [0.1,0.01,0.001,0.0001]:
        dirac_delta_vol = dirac_delta_implied_vol(call_price,S,K,T,r,q,call_sigma_bounds[0],call_sigma_bounds[1],epsilon)
        print('epsilon: {:<9}, vol: {:.2f}, dirac_delta_vol {:.4f}'.format(epsilon,sigma,dirac_delta_vol))
    

###################################
Static Parameters
###################################
sigma: 0.5
r:     -0.01
q:     0.01
K:     200
T:     0.5
###################################
$$$$$$$$$$$$$$$$$$$$$
Moneyness S/K=0.4
$$$$$$$$$$$$$$$$$$$$$
S:     80.00
call:  0.06
put:   121.46
lower_vol: 0.0026972877882851733
upper_vol: 1.6195221025849988
epsilon: 0.1      , vol: 0.50, dirac_delta_vol 0.3473
epsilon: 0.01     , vol: 0.50, dirac_delta_vol 0.3688
epsilon: 0.001    , vol: 0.50, dirac_delta_vol 0.4732
epsilon: 0.0001   , vol: 0.50, dirac_delta_vol 0.5193
$$$$$$$$$$$$$$$$$$$$$
Moneyness S/K=0.7
$$$$$$$$$$$$$$$$$$$$$
S:     140.00
call:  4.54
put:   66.24
lower_vol: 0.11564630045059643
upper_vol: 0.7459388464124055
epsilon: 0.1      , vol: 0.50, dirac_delta_vol 0.5083
epsilon: 0.01     , vol: 0.50, dirac_delta_vol 0.5086
epsilon: 0.001    , vol: 0.50, dirac_delta_vol 0.5086
epsilon: 0.0001   , vol: 0.50, dirac_delta_vol 0.0000
$$$$$$$$$$$$$$$$$$$$$
Moneyness S/K=0.8
$$$$$$$$$$$$$$$$$$

## Conclusion
Simple in form and good to know a close-form model-free formula on implied volatility. Indeed, getting the implied volatility from an arbitrary parameterized model is valuable especially in deriative's trading. Investment banks tend to use their own models in valuation but ultimately would be interpreted as implied vol and compare it market bid/offer from rivals. So this framework should be a long-last and forever topic in Financial Engineering.

### On clarity:
The implied volatility barrier formula (Lemma 1) is only for **call option case**. The put case boundaries can be addressed easily by call-put parity but better to claim it explicitly

### On boundary estimation
The formula is simple (easy to implement) but not very tight, do we have better estimates on that?

### On numerical implementation

Because of epsilon and Dirac Delta function, it requires high precision in integration (finer steps), otherwise would not be easy to get stable solutions. See following cases
1. early expiry
2. too small epsilon would return zero without using sophisticated numerical integration (step should adaptive to epsilon)
3. deep ITM or OTM
Therefore, may be practical to elaborate how to numerically solve the integration part.